In [ ]:
# Do not delete this file. It is used to set environment variables for PySpark in Jupyter Notebooks.
# Helper script to set PySpark environment variables for Jupyter Notebooks
import os

# Define a function to set PySpark environment variables
def set_pyspark_env_vars():
    """
    Sets the PYSPARK_PYTHON and PYSPARK_DRIVER_PYTHON environment variables.
    This is necessary to help Spark find the correct Python interpreter.
    You MUST replace the path below with the absolute path to your python.exe.
    """
    os.environ['PYSPARK_PYTHON'] = "path_to_your_python_executable"
    os.environ['PYSPARK_DRIVER_PYTHON'] = "path_to_your_python_executable"
    print("PySpark environment variables set.")

set_pyspark_env_vars()

In [ ]:
#define a helper function to get the path of the project directory
import os
def get_notebook_path():
        try:
        # This works for scripts run from the command line
            SCRIPT_DIR = os.path.dirname(os.path.abspath(__file__))
        except NameError:
            # This is the dynamic way for Jupyter Notebooks
            SCRIPT_DIR = os.path.abspath(os.getcwd())
        return SCRIPT_DIR
    
# Get the base path to your Hudi table 
SCRIPT_DIR = get_notebook_path()
HUDI_BASE_PATH = os.path.join(SCRIPT_DIR, "hudi_copy_on_write_table_data")
print(f"Notebook path: {SCRIPT_DIR}")
print(f"Hudi base path: {HUDI_BASE_PATH}")  
    

In [ ]:
# Helpers to create a Spark session with Hudi jars


def create_spark_session():
    print("Creating Spark session with Hudi jars...")
    # Import necessary libraries
    
    import os
    import findspark

    # Findspark helps to find the Spark installation on your system
    # You might need to add this line if it's not already in your path
    findspark.init(spark_home=os.environ.get("SPARK_HOME"))
    from pyspark.sql import SparkSession

    # Path to the directory containing Hudi jar files
    jars_path = os.path.join(SCRIPT_DIR, "jars")
    print(f"Resolved jars path: {jars_path}")

    hudi_jars = [
            os.path.join(jars_path, 'hudi-spark3.4-bundle_2.12-0.14.1.jar')
        ]
    print(f"Hudi jars will be loaded from: {hudi_jars}")
    print(f"Jars path resolved to: {jars_path}")
    
    # Configure Spark session with Hudi configurations and jars
    # Adjust Spark configurations as necessary
    spark = SparkSession.builder \
        .appName("HudiExplore-TripApp-MergeOnRead") \
        .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.hudi.catalog.HoodieCatalog") \
        .config("spark.sql.extensions", "org.apache.spark.sql.hudi.HoodieSparkSessionExtension") \
        .config("spark.kryo.registrator", "org.apache.spark.HoodieSparkKryoRegistrar") \
        .config("spark.jars", ",".join(hudi_jars)) \
        .getOrCreate()
    print("Spark session with Hudi jars has been created.")
    return spark
create_spark_session()


In [ ]:
# Define a function to create sample data for this example
# Adjust the schema and data as necessary for your use case

import time, datetime
from datetime import datetime

def create_sample_data(number_of_records):     # Schema for ride-sharing data

    # Generate large dataset (simulate 100 trips)
    # The ts field should be a proper datetime object for TimestampType
        
    data = [(f"trip_{i}", datetime.fromtimestamp(time.time() + i), f"rider_{i% 1000}", f"driver_{i%500}", 20.0 + (i%50), 
            ["nyc", "sf", "la"][i%3], f"2025-09-{(i%30)+1:02}") for i in range(number_of_records)]
    # print data --- IGNORE ---
    print(data)
    return data

#create_sample_data(100)

In [ ]:
# Define a function to create Spark dataframe from sample data

from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, TimestampType
from pyspark.sql.functions import to_date
from pyspark.sql import SparkSession

def create_dataframe(spark, data):
    # Schema for ride-sharing data
    schema = StructType([
        StructField("trip_id", StringType(), False),
        StructField("ts", TimestampType(), True),
        StructField("rider_id", StringType(), True),
        StructField("driver_id", StringType(), True),
        StructField("fare", DoubleType(), True),
        StructField("city", StringType(), True),
        StructField("trip_date", StringType(), True)  # For partitioning
    ])
    
    df = spark.createDataFrame(data, schema)
    # Convert trip_date string to actual date type
    # For partitioning, Hudi expects the partition field to be of date type if it's a date
    df = df.withColumn("trip_date", to_date(df["trip_date"], "yyyy-MM-dd")) 
    print("Sample DataFrame created:")
    df.show(5)
    return df

# get or create Spark session
spark = SparkSession.getActiveSession()
if spark is None:
    print("No active SparkSession found. Please run your setup cell first.")
    spark=create_spark_session()

#create_dataframe(spark, create_sample_data(100))

In [ ]:

# Helper function to delete existing Hudi table directory if it exists
# This ensures a clean state for the example
import shutil
def cleanup_hudi_table():
    try:
        SCRIPT_DIR = os.path.dirname(os.path.abspath(__file__))
        HUDI_TABLE_NAME = "hudi_MERGE_ON_READ"
        HUDI_BASE_PATH = os.path.join(SCRIPT_DIR, "hudi_mor_table")
        if os.path.exists(HUDI_BASE_PATH):
            print(f"Cleaning up existing data at {HUDI_BASE_PATH}")
            shutil.rmtree(HUDI_BASE_PATH)
        else:
            print(f"Directory {HUDI_BASE_PATH} does not exist, no cleanup needed.")
    except NameError:
        SCRIPT_DIR = os.path.abspath(os.getcwd())

#cleanup_hudi_table()



In [ ]:
# Define a function to get Hudi parameters
import os


def get_hudi_param():
    print("Initializing Hudi parameters...")
   
    try:
        SCRIPT_DIR = os.path.dirname(os.path.abspath(__file__))
    except NameError:
        # Fallback to the current working directory if __file__ is not defined
        SCRIPT_DIR = os.path.abspath(os.getcwd())

    # Define the Hudi table name and base path
    HUDI_TABLE_NAME = "hudi_trips_table_mor"
    HUDI_BASE_PATH = os.path.join(SCRIPT_DIR, "hudi_table_data_mor")

    # Define Hudi parameters

    hudi_options = {
        'hoodie.table.name': HUDI_TABLE_NAME,
        'hoodie.datasource.write.recordkey.field': "trip_id",
        'hoodie.datasource.write.partitionpath.field': "city",
        'hoodie.datasource.write.precombine.field': "ts",
        'hoodie.datasource.write.operation': 'upsert',
        'hoodie.upsert.shuffle.parallelism': 2,
        'hoodie.insert.shuffle.parallelism': 2
    }

    print(f"Hudi parameters initialized: {hudi_options}")
    return hudi_options, HUDI_BASE_PATH

# Call this function
get_hudi_param()

In [ ]:
# Define a function to perform the initial bulk insert to Hudi table with Merge-On-Read storage
# Note: The table type can be either COPY_ON_WRITE or MERGE_ON_READ. Here we use MERGE_ON_READ for simplicity.
import sys
from pyspark.sql.functions import monotonically_increasing_id

def bulk_insert_hudi(spark, hudi_options_base, HUDI_BASE_PATH):
    print("Performing bulk insert to Hudi table...")
    # Create a DataFrame with new data for bulk insert
    bulk_data = create_sample_data(50)  # Create 50 new sample records
    bulk_df = create_dataframe(spark, bulk_data)
    # Map the generated columns to the Hudi schema
    bulk_df = bulk_df.withColumn("uuid", monotonically_increasing_id().cast(StringType()))
    bulk_df = bulk_df.withColumnRenamed("rider_id", "rider")

    # Set the table type to Copy-on-Write for the bulk insert
    hudi_options_bulk = hudi_options_base.copy()
    hudi_options_bulk['hoodie.datasource.write.table.type'] = 'MERGE_ON_READ'
    hudi_options_bulk['hoodie.datasource.write.operation'] = 'bulk_insert'
    
    print("Writing bulk inserted data to Hudi table...")
    bulk_df.write.format("hudi") \
        .options(**hudi_options_bulk) \
        .mode("append") \
        .save(HUDI_BASE_PATH)
    print("Hudi table bulk inserted successfully.")
    hudi_read_df = spark.read.format("hudi").load(HUDI_BASE_PATH)  
    hudi_read_df.show()
    return hudi_read_df


# get or create Spark session
spark = SparkSession.getActiveSession()
if spark is None:
    print("No active SparkSession found. Please run your setup cell first.")
    spark=create_spark_session()

# Get Hudi parameters and the base path
hudi_options_base, hudi_base_path = get_hudi_param()

# Call the bulk insert function
# Perform the bulk insert
bulk_insert_hudi(spark, hudi_options_base, hudi_base_path)


The Hudi table on disk carries the schema from its first creation. When you perform an upsert with new data, Hudi performs the following steps:

Reads the Existing Schema: It checks the schema of the Hudi table already saved on disk.

Aligns the New Data: It compares the schema of your new upsert_df with the existing table schema.

Applies Schema Evolution: If the data does not contain a specific column, Hudi correctly recognizes this and automatically adds the  column to the incoming data with null values to match the existing table schema.

This Schema Evolution behavior is by design. It allows to add or drop columns over time without having to rebuild the entire table, making schema changes much more flexible.

In [ ]:

# Define a function to do incremental updates to Hudi table with Merge-On-Read storage in Upsert mode



import time, datetime
from datetime import datetime
from pyspark.sql.functions import lit

def upsert_hudi(spark, hudi_options_base, HUDI_BASE_PATH):
    print("Performing upsert update to Hudi table...")
    # Create a DataFrame with a new data and one updated data
    upsert_data = [
        ("trip_2", datetime.fromtimestamp(time.time() + 101), "rider_2_updated", "driver_2", 100.0, "la", "2025-09-03"),
        ("trip_101", datetime.fromtimestamp(time.time() + 102), "rider_101", "driver_101", 200.0, "nyc", "2025-09-01")
    ]
    upsert_schema = StructType([
        StructField("trip_id", StringType(), False),
        StructField("ts", TimestampType(), True),
        StructField("rider", StringType(), True),
        StructField("driver_id", StringType(), True),
        StructField("fare", DoubleType(), True),
        StructField("city", StringType(), True),
        StructField("trip_date", StringType(), True)
    ])
    upsert_df = spark.createDataFrame(upsert_data, upsert_schema)
    upsert_df = upsert_df.withColumn("uuid", lit(None).cast(StringType())) # We'll let Hudi determine the uuid for new records
    
    # Set the table type to Copy-on-Write for the upsert
    hudi_options_upsert = hudi_options_base.copy()
    hudi_options_upsert['hoodie.datasource.write.table.type'] = 'COPY_ON_WRITE'
    hudi_options_upsert['hoodie.datasource.write.operation'] = 'upsert'
    print("Writing upserted data to Hudi table...")
    upsert_df.write.format("hudi") \
        .options(**hudi_options_upsert) \
        .mode("append") \
        .save(HUDI_BASE_PATH)
    print("Hudi table upserted successfully.")
    hudi_read_df = spark.read.format("hudi").load(HUDI_BASE_PATH)
    hudi_read_df.show()
    return hudi_read_df

# get or create Spark session
spark = SparkSession.getActiveSession()
if spark is None:
    print("No active SparkSession found. Please run your setup cell first.")
    spark=create_spark_session()
# Get Hudi parameters and the base path
hudi_options_base, hudi_base_path = get_hudi_param()


# Call the upsert update function
upsert_hudi(spark, hudi_options_base, hudi_base_path)


Now, that the initial bulk load is done using Copy On Write storage, lets get to the heart of the Apache Hudi. 
The distinction between snapshot and read-optimized queries is crucial, especially when deciding how to access the upsert data.

Snapshot Query
A snapshot query provides the most up-to-date view of a Hudi table. It reads the latest committed state of the data, ensuring you get a complete and accurate "snapshot" of the table at a specific point in time.

How it works (for a Copy on Write table): With a Copy on Write (CoW) table, every update or insert operation creates a new version of the Parquet base file. A snapshot query simply reads all the latest versions of these Parquet files. There are no incremental files to merge, so the query is straightforward and efficient.

When to choose it: This is the default and recommended choice for most use cases with CoW tables. It's the simplest way to read the current state of your data. The code spark.read.format("hudi").load(HUDI_BASE_PATH) performs a snapshot query by default.

Read-Optimized Query
A read-optimized query is designed to be as fast as possible by reading only the Parquet base files, completely ignoring any recent incremental data.

How it works (for a Copy on Write table): Since a CoW table already stores all its data in Parquet base files, a read-optimized query behaves exactly the same as a snapshot query. It reads the same set of files and provides the identical result. You will not see a performance difference between the two for a CoW table.

When to choose it: This query type is primarily useful for Merge on Read (MoR) tables, not Copy on Write. With a MoR table, a read-optimized query would be faster than a snapshot query because it skips the step of merging the Parquet base files with the Avro/ORC delta logs.  However, this speed comes at the cost of having potentially stale data, as it won't include recent updates that are only in the delta logs.




In [ ]:
# This function will do a snapshot query on the Hudi table and display the results.
# In a Merge-On-Read (MoR) table, data is stored in a combination of columnar base files and row-based log files.
# When performing a snapshot query, Hudi automatically merges the base files with the log files to present the latest view of the data.
# The merge happens on the fly to present with the most up-to-date data.
# This ensures full data freshness but can be slower due to the merging overhead.
# This means you get a fast, efficient query that's ideal for a read-heavy workload.
# Merge-On-Read (MoR) tables are designed to handle high write throughput and provide efficient storage for large datasets.
# The downside is that the results may not include the most recent data because the latest updates and inserts are still sitting in the log files.


def snapshot_query_hudi(spark, HUDI_BASE_PATH):
    print("Performing snapshot query on Hudi table...")
    trip_snapshot_df = spark.read.format("hudi").load(HUDI_BASE_PATH)
    print("Snapshot DataFrame loaded from Hudi table:")
    trip_snapshot_df.show(5)
    print("Creating temporary view for SQL queries...")
    trip_snapshot_df.createOrReplaceTempView("hudi_trips_snapshot")
    print("Snapshot query result:")
    spark.sql("SELECT * FROM hudi_trips_snapshot ORDER BY ts DESC").show(10)
    spark.sql("SELECT city, COUNT(*) as trip_count FROM hudi_trips_snapshot GROUP BY city ORDER BY trip_count DESC").show()
    # Saving this to a df for further use if needed
    hudi_snapshot_df = spark.sql("SELECT * FROM hudi_trips_snapshot ORDER BY ts DESC")
    hudi_snapshot_df.show(5)
    return trip_snapshot_df

# get or create Spark session
spark = SparkSession.getActiveSession()
if spark is None:
    print("No active SparkSession found. Please run your setup cell first.")
    spark=create_spark_session()

# Get Hudi parameters and the base path
hudi_options_base, hudi_base_path = get_hudi_param()

# Call the snapshot query function
snapshot_query_hudi(spark, hudi_base_path)

In [ ]:
# This funtion will do a read-optimized query on the Hudi table to show the latest state of the data
# In a Merge-On-Read (MoR) table, data is stored in a combination of columnar base files and row-based log files.
# When performing a read-optimized query, Hudi reads only the columnar base files without merging in the log files.
# This makes read-optimized queries very fast and efficient, as they can leverage the optimized columnar storage format.
# The output is exactly the same as the default spark.read.format("hudi").load(HUDI_BASE_PATH)
# This makes read-optimized queries very fast and efficient, as they can leverage the optimized columnar storage format.
# The downside is that the results may not include the most recent data because the latest updates and inserts are still sitting in the log files.  


def read_optimized_query_hudi(spark, HUDI_BASE_PATH):
    print("Performing read-optimized query on Hudi table...")
    trip_ro_df = spark.read.format("hudi").option("hoodie.datasource.query.type", "read_optimized").load(HUDI_BASE_PATH)
    print("Read-Optimized DataFrame loaded from Hudi table:")
    trip_ro_df.show(5)
    print("Creating temporary view for SQL queries...")
    trip_ro_df.createOrReplaceTempView("hudi_trips_ro")
    print("Read-Optimized query result:")
    spark.sql("SELECT * FROM hudi_trips_ro ORDER BY ts DESC").show(10)
    spark.sql("SELECT city, COUNT(*) as trip_count FROM hudi_trips_ro GROUP BY city ORDER BY trip_count DESC").show()
    # Saving this to a df for further use if needed
    hudi_ro_df = spark.sql("SELECT * FROM hudi_trips_ro ORDER BY ts DESC")
    hudi_ro_df.show(5)
    return trip_ro_df

# get or create Spark session
spark = SparkSession.getActiveSession()
if spark is None:
    print("No active SparkSession found. Please run your setup cell first.")
    spark=create_spark_session()

# Get Hudi parameters and the base path
hudi_options_base, hudi_base_path = get_hudi_param()

# Call the read optimized function
read_optimized_query_hudi(spark,hudi_base_path)

In [ ]:
# Define a function to create new commits and records in the Hudi table

import time, datetime
from datetime import datetime
from pyspark.sql.functions import lit, col, to_date
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, DoubleType, IntegerType, BooleanType
import sys
import os


def upsert_schema_eval(spark, hudi_options_base, HUDI_BASE_PATH):
    
    #print existing table schema
    print("Existing Hudi table schema:")
    spark.read.format("hudi").load(HUDI_BASE_PATH).printSchema()
    print("Creating new records")
    
    #upsert_df=create_dataframe(spark, create_sample_data(5))
    # Add a new column 'trip_type' with values "Personal" or "Business" the for new records
    #upsert_df = upsert_df.withColumn("trip_type", lit("Personal"))
    # Add a new column 'payment_method' with values "Credit Card", "Cash", or "Digital Wallet" the for new records
    #upsert_df = upsert_df.withColumn("payment_method", lit("Credit Card"))
    # Add a new column 'is_peak_hour' with boolean values indicating if the trip was during peak hours (e.g., 7-9 AM, 5-7 PM)
    #upsert_df = upsert_df.withColumn("is_peak_hour", lit(True)) 
    # Add a new column 'trip_duration' with random integer values representing the trip duration in minutes
    #upsert_df = upsert_df.withColumn("trip_duration", lit(15))
    # Add a new column 'trip_distance' with random float values representing the trip distance in miles
    #upsert_df = upsert_df.withColumn("trip_distance", lit(5.5))
    # Add a new column 'payment_status' with values "Paid", "Pending", or "Failed"
    #upsert_df = upsert_df.withColumn("payment_status", lit("Paid"))


  # Create new records with the additional columns
    new_records = [
        ("trip_100", datetime.fromtimestamp(time.time()), "rider_100", "driver_100", 25.0, "la", "2025-09-03", "Personal", "Credit Card", True, 15, 5.5, "Paid"),
        ("trip_101", datetime.fromtimestamp(time.time()), "rider_101", "driver_101", 30.0, "sf", "2025-09-04", "Business", "Digital Wallet", False, 30, 10.0, "Pending")
    ]
    new_schema = StructType([
        StructField("trip_id", StringType(), False),
        StructField("ts", TimestampType(), True),
        StructField("rider", StringType(), True),
        StructField("driver_id", StringType(), True),
        StructField("fare", DoubleType(), True),
        StructField("city", StringType(), True),
        StructField("trip_date", StringType(), True),
        StructField("trip_type", StringType(), True),
        StructField("payment_method", StringType(), True),
        StructField("is_peak_hour", BooleanType(), True),
        StructField("trip_duration", IntegerType(), True),
        StructField("trip_distance", DoubleType(), True),
        StructField("payment_status", StringType(), True)
    ])
    # Create a single DataFrame from all new records
    upsert_df = spark.createDataFrame(new_records, new_schema)
    
    # Cast the trip_date column to the correct DateType to match the existing table schema
    upsert_df = upsert_df.withColumn("trip_date", to_date(col("trip_date"), "yyyy-MM-dd"))
    
    # Add the uuid column as required by Hudi
    upsert_df = upsert_df.withColumn("uuid", lit(None).cast(StringType()))
    
    print("New records with additional columns:")
    upsert_df.show()
    print("New schema with additional columns:")
    upsert_df.printSchema()



    # Set the table type to Merge-on-Read and enable schema evolution
    hudi_options_upsert = hudi_options_base.copy()
    hudi_options_upsert['hoodie.datasource.write.table.type'] = 'MERGE_ON_READ'
    hudi_options_upsert['hoodie.datasource.write.operation'] = 'upsert'
    hudi_options_upsert['hoodie.datasource.write.schema.allow.auto.evolution'] = 'true'
    hudi_options_upsert['hoodie.datasource.write.schema.on-read.enable'] = 'true'

    print("Writing upserted data to Hudi table...")
    upsert_df.write.format("hudi") \
        .options(**hudi_options_upsert) \
        .mode("append") \
        .save(HUDI_BASE_PATH)
    print("Hudi table upserted successfully.")
    hudi_read_df = spark.read.format("hudi").load(HUDI_BASE_PATH)
    hudi_read_df.show()
    return hudi_read_df



# get or create Spark session
spark = SparkSession.getActiveSession()
if spark is None:
    print("No active SparkSession found. Please run your setup cell first.")
    spark=create_spark_session()
    
# Get Hudi parameters and the base path
hudi_options_base, hudi_base_path = get_hudi_param()

# Call the batch function
upsert_schema_eval(spark, hudi_options_base, hudi_base_path)

    

In [ ]:


def incremental_query_hudi(spark, hudi_base_path):
    """
    Performs an incremental query on the Hudi table to fetch only the new data.
    
    This function reads the latest commit instant time from the Hudi timeline
    and uses the second-to-last commit as the starting point for the incremental query.
    
    """
    print("Reading table for latest commit time for incremental query...")
    # Get the latest commit time from the Hudi table
    commits = list(map(lambda row: row[0],
                       spark.read.format("hudi")
                       .load(hudi_base_path)
                       .select("_hoodie_commit_time")
                       .distinct()
                       .orderBy("_hoodie_commit_time")
                       .collect()))
    
    if not commits:
        print("No commits found. Cannot perform incremental query.")
        return
    if len(commits) < 2:
        print("Not enough commits to perform incremental query from second-to-last commit.")
        print(f"Commits found: {commits}")
        return
    #latest_commit = commits[len(commits) - 1]
    second_to_last_commit = commits[len(commits) - 2]
    print(f"Second-to-last commit instant: {second_to_last_commit}")
    
    
    print(f"Latest commit time for incremental query: {second_to_last_commit}")
    
    # Perform the incremental query
    incremental_query_options = {
        'hoodie.datasource.query.type': 'incremental',
        'hoodie.datasource.query.incremental.pull_style': 'full_scan',
        'hoodie.datasource.read.begin.instanttime': second_to_last_commit
    }

    print("Executing incremental query on Hudi table...")
    incremental_df = spark.read.format("hudi") \
        .options(**incremental_query_options) \
        .load(hudi_base_path)

    print("Incremental query results:")
    incremental_df.show()
    
    return incremental_df

# get or create Spark session
spark = SparkSession.getActiveSession()
if spark is None:
    print("No active SparkSession found. Please run your setup cell first.")
    spark=create_spark_session()
# Get Hudi parameters and the base path
hudi_options_base, hudi_base_path = get_hudi_param()
# Call the incremental query function
incremental_query_hudi(spark, hudi_base_path)

In [ ]:
# Define a function to hard delete records from Hudi table
def hard_delete_hudi(spark, hudi_options_base, HUDI_BASE_PATH):
    print("Performing hard delete on Hudi table...")
    # Create a DataFrame with the record keys to delete
    delete_data = [
        ("trip_3",),  # Assuming trip_3 exists
        ("trip_50",)  # Assuming trip_50 exists
    ]
    delete_schema = StructType([
        StructField("trip_id", StringType(), False)
    ])
    delete_df = spark.createDataFrame(delete_data, delete_schema)
    
    # Set the operation to 'delete'
    hudi_options_delete = hudi_options_base.copy()
    hudi_options_delete['hoodie.datasource.write.operation'] = 'delete'
    
    print("Writing delete records to Hudi table...")
    delete_df.write.format("hudi") \
        .options(**hudi_options_delete) \
        .mode("append") \
        .save(HUDI_BASE_PATH)
    print("Hudi table hard delete completed.")
    hudi_read_df = spark.read.format("hudi").load(HUDI_BASE_PATH)
    hudi_read_df.show()
    return hudi_read_df

# get or create Spark session
spark = SparkSession.getActiveSession()
if spark is None:
    print("No active SparkSession found. Please run your setup cell first.")
    spark=create_spark_session()
# Get Hudi parameters and the base path
hudi_options_base, hudi_base_path = get_hudi_param()
# Call the hard delete function
hard_delete_hudi(spark, hudi_options_base, hudi_base_path)

In [ ]:
# Define a function to do a new indexed read query on the Hudi table
def indexed_read_query_hudi(spark, HUDI_BASE_PATH):
    print("Performing indexed read query on Hudi table...")
    trip_indexed_df = spark.read.format("hudi").option("hoodie.datasource.query.type", "snapshot").load(HUDI_BASE_PATH)
    print("Indexed Read DataFrame loaded from Hudi table:")
    trip_indexed_df.show(5)
    print("Creating temporary view for SQL queries...")
    trip_indexed_df.createOrReplaceTempView("hudi_trips_indexed")
    print("Indexed read query result:")
    spark.sql("SELECT * FROM hudi_trips_indexed ORDER BY ts DESC").show(10)
    spark.sql("SELECT city, COUNT(*) as trip_count FROM hudi_trips_indexed GROUP BY city ORDER BY trip_count DESC").show()
    # Saving this to a df for further use if needed
    hudi_indexed_df = spark.sql("SELECT * FROM hudi_trips_indexed ORDER BY ts DESC")
    hudi_indexed_df.show(5)
    return trip_indexed_df

# get or create Spark session
spark = SparkSession.getActiveSession()
if spark is None:
    print("No active SparkSession found. Please run your setup cell first.")
    spark=create_spark_session()

# Get Hudi parameters and the base path
hudi_options_base, hudi_base_path = get_hudi_param()
# Call the indexed read query function
indexed_read_query_hudi(spark, hudi_base_path)
# This function will clean up the Hudi table data directory

In [ ]:
# Define a function to show time travel query on the Hudi table
def time_travel_query_hudi(spark, HUDI_BASE_PATH):
    print("Performing time travel query on Hudi table...")
    
    # Read all commits and find the earliest one
    # Note: This loads metadata, not the full dataset, so it's efficient.
    try:
        commits = spark.read.format("hudi").load(HUDI_BASE_PATH).select("_hoodie_commit_time").distinct().collect()
        if not commits:
            print("No commits found in the table. Cannot perform time travel query.")
            return
        
        # Sort commits and get the earliest one
        commit_times = sorted([row["_hoodie_commit_time"] for row in commits])
        print(f"Available commits: {commit_times}")
        
    except NameError as e:
        print(f"Error reading commits. Make sure the Hudi table exists at: {HUDI_BASE_PATH}")
        print(e)
        return

    # For demonstration, we'll pick the second commit if available
    if len(commit_times) < 2:
        print("Not enough commits to perform time travel. Need at least 2 commits.")
        return
    
    travel_time = commit_times[1]
    print(f"Time travel to commit: {travel_time}")

    # Perform the time travel query using the selected commit time
    time_travel_df = spark.read.format("hudi") \
        .option("as.of.instant", travel_time) \
        .load(HUDI_BASE_PATH)
    print("Time travel query DataFrame loaded.")
    time_travel_df.show(5)
    # Run a sample SQL query on the time-traveled data
    time_travel_df.createOrReplaceTempView("hudi_trips_time_travel")
    print("Time travel query result:")
    spark.sql("SELECT trip_id, fare, ts FROM hudi_trips_time_travel ORDER BY ts DESC").show(10)
    print("Time travel query completed.")
    return time_travel_df

# get or create Spark session
spark = SparkSession.getActiveSession()
if spark is None:
    print("No active SparkSession found. Please run your setup cell first.")
    spark=create_spark_session()
# Get Hudi parameters and the base path
hudi_options_base, hudi_base_path = get_hudi_param()
# Call the time travel query function
time_travel_query_hudi(spark, hudi_base_path)
